<a href="https://colab.research.google.com/github/masoudshahrian/DeepLearning-Code/blob/main/license_plate_detector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [31]:
!sudo apt-get install tesseract-ocr -y
!pip install pytesseract
!pip install easyocr
!pip install opencv-python

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2.1build1).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.


In [43]:
# that's a great code do not ommite it

# Import required libraries
import cv2
import easyocr
from PIL import Image
import os
import re
import numpy as np
import logging
from datetime import datetime
import pandas as pd

# Set up basic logging for Kaggle environment
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)

def setup_kaggle_paths():

    # Kaggle input directory
    INPUT_DIR = '/content/'
    # Kaggle working directory for outputs
    OUTPUT_DIR = '.'

    os.makedirs(OUTPUT_DIR, exist_ok=True)

    return INPUT_DIR, OUTPUT_DIR

def load_and_preprocess_image(image_path):
    """Load and preprocess the image for OCR"""
    try:
        # Load image using PIL
        image_pil = Image.open(image_path)
        # Convert to OpenCV format
        image_cv = cv2.cvtColor(np.array(image_pil), cv2.COLOR_RGB2BGR)

        # Preprocessing steps optimized for Kaggle
        gray = cv2.cvtColor(image_cv, cv2.COLOR_BGR2GRAY)
        denoised = cv2.fastNlMeansDenoising(gray, h=10)
        thresh = cv2.adaptiveThreshold(
            denoised,
            255,
            cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
            cv2.THRESH_BINARY,
            11,
            2
        )

        # Save preprocessed image for visualization in Kaggle notebook
        cv2.imwrite('preprocessed_image.png', thresh)
        logging.info("Image preprocessing completed and saved")
        return thresh

    except Exception as e:
        logging.error(f"Error processing image: {e}")
        return None

def extract_text_from_image(preprocessed_image, reader):
    """Extract text from the preprocessed image"""
    try:
        # Perform OCR with progress tracking for Kaggle
        logging.info("Starting OCR process...")
        results = reader.readtext(
            preprocessed_image,
            detail=1,  # Get detailed output for better analysis
            allowlist='abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789'
        )

        # Extract text and confidence scores
        texts = []
        confidences = []
        boxes = []

        for result in results:
            bbox, text, confidence = result
            texts.append(text)
            confidences.append(confidence)
            boxes.append(bbox)

        # Create DataFrame for Kaggle visualization
        ocr_df = pd.DataFrame({
            'text': texts,
            'confidence': confidences,
            'bounding_box': boxes
        })

        # Save OCR results to CSV for further analysis
        ocr_df.to_csv('ocr_results.csv', index=False)

        return '\n'.join(texts), ocr_df

    except Exception as e:
        logging.error(f"Error extracting text: {e}")
        return "", pd.DataFrame()

def find_words_near_digits(text):
    """Find words that appear near digits in the text"""
    try:
        lines = text.split('\n')
        results = []

        pattern = r'([a-zA-Z0-9]*[a-zA-Z][a-zA-Z0-9]*)\s*(\d+)|(\d+)\s*([a-zA-Z0-9]*[a-zA-Z][a-zA-Z0-9]*)'

        for line in lines:
            matches = re.findall(pattern, line)
            for match in matches:
                word = match[0] or match[3]
                digit = match[1] or match[2]
                results.append((word, digit))

        # Convert results to DataFrame for Kaggle visualization
        results_df = pd.DataFrame(results, columns=['Word', 'Digit'])
        results_df.to_csv('word_digit_pairs.csv', index=False)

        return results, results_df

    except Exception as e:
        logging.error(f"Error finding word-digit pairs: {e}")
        return [], pd.DataFrame()

def save_results(extracted_text, word_digit_pairs, ocr_df, output_dir):
    """Save results in formats suitable for Kaggle"""
    try:
        # Save text results
        with open(os.path.join(output_dir, 'output.txt'), 'w', encoding='utf-8') as f:
            f.write(f"Analysis Date (UTC): {datetime.utcnow().strftime('%Y-%MM-%DD %H:%M:%S')}\n\n")
            f.write("Extracted Text:\n")
            f.write(extracted_text + "\n\n")
            f.write("Word-Digit Pairs:\n")
            for word, digit in word_digit_pairs:
                f.write(f"Word: {word}, Digit: {digit}\n")

        # Save analysis summary
        summary_df = pd.DataFrame({
            'Metric': ['Total Words', 'Total Digit Pairs', 'Average Confidence'],
            'Value': [
                len(extracted_text.split()),
                len(word_digit_pairs),
                ocr_df['confidence'].mean() if not ocr_df.empty else 0
            ]
        })
        summary_df.to_csv(os.path.join(output_dir, 'analysis_summary.csv'), index=False)

        logging.info("Results saved successfully")

    except Exception as e:
        logging.error(f"Error saving results: {e}")

def main():
    # Setup Kaggle environment
    INPUT_DIR, OUTPUT_DIR = setup_kaggle_paths()

    # Initialize EasyOCR with GPU support (Kaggle provides GPU)
    reader = easyocr.Reader(['en'], gpu=True)

    # Process image (adjust path according to your Kaggle dataset structure)
    image_path = os.path.join(INPUT_DIR, 'f1.png')

    # Process image
    preprocessed_image = load_and_preprocess_image(image_path)
    if preprocessed_image is None:
        print("Failed to load or preprocess image")
        return

    # Extract text with detailed analysis
    extracted_text, ocr_df = extract_text_from_image(preprocessed_image, reader)
    if not extracted_text:
        print("No text extracted from image")
        return

    # Find and analyze word-digit pairs
    word_digit_pairs, pairs_df = find_words_near_digits(extracted_text)

    # Display results (will be shown in Kaggle notebook)
    print("OCR Analysis Results:")
    print("-" * 50)
    print("\nExtracted Text:")
    print(extracted_text)
    print("\nWord-Digit Pairs:")
    print(pairs_df)
    print("\nOCR Confidence Scores:")
    print(ocr_df['confidence'].describe())

    # Save all results
    save_results(extracted_text, word_digit_pairs, ocr_df, OUTPUT_DIR)

    # Return DataFrames for further analysis in Kaggle notebook
    return ocr_df, pairs_df

# # For Kaggle notebook execution
# if __name__ == "__main__":
#     ocr_results, word_digit_results = main()

In [44]:
def main():
    # Setup Kaggle environment
    INPUT_DIR, OUTPUT_DIR = setup_kaggle_paths()

    # Initialize EasyOCR with GPU support (Kaggle provides GPU)
    reader = easyocr.Reader(['en'], gpu=True)

    # Process image (adjust path according to your Kaggle dataset structure)
    image_path = os.path.join(INPUT_DIR, 'pl1.jpg')

    # Process image
    preprocessed_image = load_and_preprocess_image(image_path)
    if preprocessed_image is None:
        print("Failed to load or preprocess image")
        return pd.DataFrame(), pd.DataFrame()

    # Extract text with detailed analysis
    extracted_text, ocr_df = extract_text_from_image(preprocessed_image, reader)
    if not extracted_text:
        print("No text extracted from image")
        return pd.DataFrame(), pd.DataFrame()

    # Find and analyze word-digit pairs
    word_digit_pairs, pairs_df = find_words_near_digits(extracted_text)

    # Display results (will be shown in Kaggle notebook)
    print("OCR Analysis Results:")
    print("-" * 50)
    print("\nExtracted Text:")
    print(extracted_text)
    print("\nWord-Digit Pairs:")
    print(pairs_df)
    print("\nOCR Confidence Scores:")
    print(ocr_df['confidence'].describe())

    # Save all results
    save_results(extracted_text, word_digit_pairs, ocr_df, OUTPUT_DIR)

    # Return DataFrames for further analysis in Kaggle notebook
    return ocr_df, pairs_df

In [45]:
if __name__ == "__main__":
    results = main()
    if results is not None:
        ocr_results, word_digit_results = results
    else:
        ocr_results, word_digit_results = pd.DataFrame(), pd.DataFrame()
        print("Processing completed with errors - returned empty DataFrames")

OCR Analysis Results:
--------------------------------------------------

Extracted Text:
9JR1205
Whora

Word-Digit Pairs:
     Word Digit
0  9JR120     5

OCR Confidence Scores:
count    2.000000
mean     0.557098
std      0.381607
min      0.287261
25%      0.422180
50%      0.557098
75%      0.692017
max      0.826935
Name: confidence, dtype: float64
